## SNOWBALL Spark SQL version
#### **Notebook to create ANALYSIS layer**
##### **Creating ANALYSIS schema to create required ANALYSIS tables**


In [ ]:
%%sql
CREATE SCHEMA IF NOT EXISTS analysis;

##### **customer_contract**

In [ ]:
%%sql
DROP TABLE IF EXISTS analysis.customer_contract;
CREATE TABLE analysis.customer_contract AS
/* This Stored Procedure calculates the join month, end month, and churn month for each customer based on their revenue records*/

SELECT

    customer_level_1
    , customer_level_2
    , customer_level_3
    , customer_level_4
    , customer_level_5
    , customer_level_6
    , customer_level_7
    , customer_level_8
    , customer_level_9
    , MIN(month)                    AS customer_join_month
    , MAX(month)                    AS customer_end_month
    , DATEADD(MONTH, 1, MAX(month)) AS customer_churn_month

FROM "arr_sandbox"."core"."revenue"

WHERE
    mrr <> 0.0
GROUP BY
    customer_level_1
    , customer_level_2
    , customer_level_3
    , customer_level_4
    , customer_level_5
    , customer_level_6
    , customer_level_7
    , customer_level_8
    , customer_level_9


##### **customer_lifecycle_events**

In [ ]:
%%sql
DROP TABLE IF EXISTS analysis.customer_lifecycle_events;
CREATE TABLE analysis.customer_lifecycle_events AS
/* This stored procedure calculates lifecycle flags for customers based on their join and churn months, producing monthly, quarterly, yearly, and year-to-date indicators for new, churned, and existing customers*/

WITH get_month_difference AS (

    SELECT

        m.monthly_revenue_key
        , m.customer_level_1
        , m.customer_level_2
        , m.customer_level_3
        , m.customer_level_4
        , m.customer_level_5
        , m.customer_level_6
        , m.customer_level_7
        , m.customer_level_8
        , m.customer_level_9
        , m.month_roll
        , m.ytd_helper
        , c.customer_join_month
        , c.customer_end_month
        , c.customer_churn_month
        , DATEDIFF(MONTH, c.customer_join_month, m.month_roll)  AS customer_join_month_difference
        , DATEDIFF(MONTH, c.customer_churn_month, m.month_roll) AS customer_churn_month_difference

    FROM "arr_sandbox"."analysis"."monthly_revenue" m

    INNER JOIN "arr_sandbox"."analysis"."customer_contract" c
        ON
            m.customer_level_1 = c.customer_level_1
            AND m.customer_level_2 = c.customer_level_2
            AND m.customer_level_3 = c.customer_level_3
            AND m.customer_level_4 = c.customer_level_4
            AND m.customer_level_5 = c.customer_level_5
            AND m.customer_level_6 = c.customer_level_6
            AND m.customer_level_7 = c.customer_level_7
            AND m.customer_level_8 = c.customer_level_8
            AND m.customer_level_9 = c.customer_level_9
)

-- Calculating the flags based on customer join, end, and  churn month.

, customer_lifecycle_flags AS (

    SELECT

        monthly_revenue_key
        , customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , month_roll

        -- MONTHLY FLAGS
        , CASE
            WHEN month_roll = customer_join_month
                THEN 1
            ELSE 0
        END AS lm_customer_new_flag
        , CASE
            WHEN customer_churn_month = month_roll
                THEN 1
            ELSE 0
        END AS lm_customer_churn_flag
        , CASE
            WHEN
                month_roll > customer_join_month
                AND month_roll < customer_churn_month
                THEN 1
            ELSE 0
        END AS lm_customer_existing_flag

        -- QUARTERLY FLAGS
        , CASE
            WHEN customer_join_month_difference < 3
                THEN 1
            ELSE 0
        END AS l_3_m_customer_new_flag
        , CASE
            WHEN
                customer_churn_month_difference < 3
                AND customer_churn_month_difference >= 0
                THEN 1
            ELSE 0
        END AS l_3_m_customer_churn_flag
        , CASE
            WHEN
                customer_join_month_difference >= 3
                AND month_roll < customer_churn_month
                THEN 1
            ELSE 0
        END AS l_3_m_customer_existing_flag

        -- YEARLY FLAGS
        , CASE
            WHEN customer_join_month_difference < 12
                THEN 1
            ELSE 0
        END AS ltm_customer_new_flag
        , CASE
            WHEN
                customer_churn_month_difference < 12
                AND customer_churn_month_difference >= 0
                THEN 1
            ELSE 0
        END AS ltm_customer_churn_flag
        , CASE
            WHEN
                customer_join_month_difference >= 12
                AND month_roll < customer_churn_month
                THEN 1
            ELSE 0
        END AS ltm_customer_existing_flag

        -- YTD FLAGS
        , CASE
            WHEN customer_join_month_difference < ytd_helper
                THEN 1
            ELSE 0
        END AS ytd_customer_new_flag
        , CASE
            WHEN
                customer_churn_month_difference < ytd_helper
                AND customer_churn_month_difference >= 0
                THEN 1
            ELSE 0
        END AS ytd_customer_churn_flag
        , CASE
            WHEN
                customer_join_month_difference >= ytd_helper
                AND month_roll < customer_churn_month
                THEN 1
            ELSE 0
        END AS ytd_customer_existing_flag

    FROM get_month_difference
)

SELECT

    monthly_revenue_key AS customer_lifecycle_events_key
    , customer_level_1
    , customer_level_2
    , customer_level_3
    , customer_level_4
    , customer_level_5
    , customer_level_6
    , customer_level_7
    , customer_level_8
    , customer_level_9
    , month_roll
    , lm_customer_new_flag
    , lm_customer_churn_flag
    , lm_customer_existing_flag

    , l_3_m_customer_new_flag
    , l_3_m_customer_churn_flag
    , l_3_m_customer_existing_flag

    , ltm_customer_new_flag
    , ltm_customer_churn_flag
    , ltm_customer_existing_flag

    , ytd_customer_new_flag
    , ytd_customer_churn_flag
    , ytd_customer_existing_flag

FROM
    customer_lifecycle_flags


##### **customer_product_contract**

In [ ]:
%%sql
DROP TABLE IF EXISTS analysis.customer_product_contract;
CREATE TABLE analysis.customer_product_contract AS
/* This stored procedure calculates the start, end, and anticipated churn months for each customer-product pair based on recurring revenue data and product details*/

WITH get_product_start_end_month AS (

    SELECT
        customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , product_level_1
        , product_level_2
        , MIN(month) OVER (
            PARTITION BY customer_level_1
            , customer_level_2
            , customer_level_3
            , customer_level_4
            , customer_level_5
            , customer_level_6
            , customer_level_7
            , customer_level_8
            , customer_level_9, product_level_1
            , product_level_2
        )                                                                                     AS product_start_month
        , MAX(month) OVER (
            PARTITION BY customer_level_1
            , customer_level_2
            , customer_level_3
            , customer_level_4
            , customer_level_5
            , customer_level_6
            , customer_level_7
            , customer_level_8
            , customer_level_9, product_level_1
            , product_level_2
        )                                                                                     AS product_end_month
        , DATEADD(MONTH, 1, MAX(month) OVER (PARTITION BY customer_level_1, product_level_1)) AS product_churn_month

    FROM "arr_sandbox"."core"."revenue"
    WHERE
        mrr <> 0
)

SELECT

    customer_level_1
    , customer_level_2
    , customer_level_3
    , customer_level_4
    , customer_level_5
    , customer_level_6
    , customer_level_7
    , customer_level_8
    , customer_level_9
    , product_level_1
    , product_level_2
    , product_start_month
    , product_end_month
    , product_churn_month

FROM
    get_product_start_end_month

GROUP BY
    customer_level_1
    , customer_level_2
    , customer_level_3
    , customer_level_4
    , customer_level_5
    , customer_level_6
    , customer_level_7
    , customer_level_8
    , customer_level_9
    , product_level_1
    , product_level_2
    , product_start_month
    , product_end_month
    , product_churn_month


##### **customer_product_lifecycle_events**

In [ ]:
%%sql
DROP TABLE IF EXISTS analysis.customer_product_lifecycle_events;
CREATE TABLE analysis.customer_product_lifecycle_events AS
/* This stored procedure calculates product lifecycle flags by evaluating churn and existing status across different periods (monthly, quarterly, yearly, and year-to-date), based on revenue and customer lifecycle data. */

WITH get_churn_month_difference AS (
    -- Churn Month Difference

    SELECT

        m.monthly_revenue_key
        , m.customer_level_1
        , m.customer_level_2
        , m.customer_level_3
        , m.customer_level_4
        , m.customer_level_5
        , m.customer_level_6
        , m.customer_level_7
        , m.customer_level_8
        , m.customer_level_9
        , m.product_level_1
        , m.product_level_2
        , m.month_roll
        , m.ytd_helper
        , p.product_start_month
        , p.product_end_month
        , p.product_churn_month
        , c.lm_customer_existing_flag
        , c.l_3_m_customer_existing_flag
        , c.ltm_customer_existing_flag
        , c.ytd_customer_existing_flag
        , DATEDIFF(MONTH, p.product_start_month, m.month_roll) AS product_start_month_difference
        , DATEDIFF(MONTH, p.product_churn_month, m.month_roll) AS product_churn_month_difference

    FROM
        "arr_sandbox"."analysis"."monthly_revenue" m

    INNER JOIN
        "arr_sandbox"."analysis"."customer_product_contract" p
        ON
            m.customer_level_1 = p.customer_level_1
            AND m.customer_level_2 = p.customer_level_2
            AND m.customer_level_3 = p.customer_level_3
            AND m.customer_level_4 = p.customer_level_4
            AND m.customer_level_5 = p.customer_level_5
            AND m.customer_level_6 = p.customer_level_6
            AND m.customer_level_7 = p.customer_level_7
            AND m.customer_level_8 = p.customer_level_8
            AND m.customer_level_9 = p.customer_level_9
            AND
            m.product_level_1 = p.product_level_1
            AND m.product_level_2 = p.product_level_2

    INNER JOIN
        "arr_sandbox"."analysis"."customer_lifecycle_events" c
        ON
            m.monthly_revenue_key = c.customer_lifecycle_events_key
            AND m.month_roll = c.month_roll
            AND
            m.customer_level_1 = c.customer_level_1
            AND m.customer_level_2 = c.customer_level_2
            AND m.customer_level_3 = c.customer_level_3
            AND m.customer_level_4 = c.customer_level_4
            AND m.customer_level_5 = c.customer_level_5
            AND m.customer_level_6 = c.customer_level_6
            AND m.customer_level_7 = c.customer_level_7
            AND m.customer_level_8 = c.customer_level_8
            AND m.customer_level_9 = c.customer_level_9
)

, product_lifecycle_flags AS (

    SELECT
        monthly_revenue_key
        , customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , product_level_1
        , product_level_2
        , month_roll

        -- Monthly flags
        , CASE
            WHEN
                lm_customer_existing_flag = 1
                AND month_roll > product_start_month
                AND month_roll < product_churn_month
                THEN 1
            ELSE 0
        END AS lm_product_existing_flag
        , CASE
            WHEN
                lm_customer_existing_flag = 1
                AND month_roll = product_churn_month
                THEN 1
            ELSE 0
        END AS lm_product_churn_flag

        -- Quarterly flags
        , CASE
            WHEN
                l_3_m_customer_existing_flag = 1
                AND product_start_month_difference >= 3
                AND month_roll < product_churn_month
                THEN 1
            ELSE 0
        END AS l_3_m_product_existing_flag
        , CASE
            WHEN
                l_3_m_customer_existing_flag = 1
                AND product_churn_month_difference < 3
                AND product_churn_month_difference >= 0
                THEN 1
            ELSE 0
        END AS l_3_m_product_churn_flag

        -- Yearly flags
        , CASE
            WHEN
                ltm_customer_existing_flag = 1
                AND product_start_month_difference >= 12
                AND month_roll < product_churn_month
                THEN 1
            ELSE 0
        END AS ltm_product_existing_flag
        , CASE
            WHEN
                ltm_customer_existing_flag = 1
                AND product_churn_month_difference < 12
                AND product_churn_month_difference >= 0
                THEN 1
            ELSE 0
        END AS ltm_product_churn_flag

        -- YTD flags
        , CASE
            WHEN
                ytd_customer_existing_flag = 1
                AND product_start_month_difference >= ytd_helper
                AND month_roll < product_churn_month
                THEN 1
            ELSE 0
        END AS ytd_product_existing_flag
        , CASE
            WHEN
                ytd_customer_existing_flag = 1
                AND product_churn_month_difference < ytd_helper
                AND product_churn_month_difference >= 0
                THEN 1
            ELSE 0
        END AS ytd_product_churn_flag

    FROM get_churn_month_difference
)

SELECT

    monthly_revenue_key AS customer_product_lifecycle_events_key
    , customer_level_1
    , customer_level_2
    , customer_level_3
    , customer_level_4
    , customer_level_5
    , customer_level_6
    , customer_level_7
    , customer_level_8
    , customer_level_9
    , product_level_1
    , product_level_2
    , month_roll
    , lm_product_churn_flag
    , lm_product_existing_flag

    , l_3_m_product_churn_flag
    , l_3_m_product_existing_flag

    , ltm_product_churn_flag
    , ltm_product_existing_flag

    , ytd_product_churn_flag
    , ytd_product_existing_flag

FROM
    product_lifecycle_flags


##### **customer_product_revenue_events**

In [ ]:
%%sql
DROP TABLE IF EXISTS analysis.customer_product_revenue_events;
CREATE TABLE analysis.customer_product_revenue_events AS
/* This model calculates flags for revenue events by assessing product growth and decline across various periods (monthly, quarterly, yearly, and year-to-date), joining revenue data with customer and product lifecycle information to identify cross-sell, upsell, and downsell activities. */

WITH product_grew AS (

    SELECT

        period_revenue_key
        , customer_key
        , customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , customer_name
        , customer_region
        , product_key
        , product_level_1
        , product_level_2
        , other_key
        , month_roll
        , arr
        , arr_lm
        , arr_l_3_m
        , arr_ltm
        , arr_ytd

        --MONTHLY GREW
        , CASE
            WHEN sum_arr_lm_delta > 0 THEN 1
            ELSE 0
        END AS product_grew_monthly
        , CASE
            WHEN sum_arr_lm_delta < 0 THEN 1
            ELSE 0
        END AS product_declined_monthly
        --QUARTERLY GREW
        , CASE
            WHEN sum_arr_l_3_m_delta > 0 THEN 1
            ELSE 0
        END AS product_grew_quarterly
        , CASE
            WHEN sum_arr_l_3_m_delta < 0 THEN 1
            ELSE 0
        END AS product_declined_quarterly
        --YEARLY GREW
        , CASE
            WHEN sum_arr_ltm_delta > 0 THEN 1
            ELSE 0
        END AS product_grew_yearly
        , CASE
            WHEN sum_arr_ltm_delta < 0 THEN 1
            ELSE 0
        END AS product_declined_yearly
        --ytd GREW
        , CASE
            WHEN sum_arr_ytd_delta > 0 THEN 1
            ELSE 0
        END AS product_grew_ytd
        , CASE
            WHEN sum_arr_ytd_delta < 0 THEN 1
            ELSE 0
        END AS product_declined_ytd

    FROM "arr_sandbox"."analysis"."period_revenue"
)

, ranked_product AS (

    SELECT

        p_1.period_revenue_key
        , p_1.customer_key
        , p_1.customer_level_1
        , p_1.customer_level_2
        , p_1.customer_level_3
        , p_1.customer_level_4
        , p_1.customer_level_5
        , p_1.customer_level_6
        , p_1.customer_level_7
        , p_1.customer_level_8
        , p_1.customer_level_9
        , p_1.customer_name
        , p_1.customer_region
        , p_1.product_key
        , p_1.product_level_1
        , p_1.product_level_2
        , p_1.other_key
        , p_1.arr
        , p_1.arr_lm
        , p_1.arr_l_3_m
        , p_1.arr_ltm
        , p_1.arr_ytd
        , p_1.month_roll
        , c.lm_customer_new_flag
        , c.l_3_m_customer_new_flag
        , c.ltm_customer_new_flag
        , c.ytd_customer_new_flag
        , c.lm_customer_churn_flag
        , c.l_3_m_customer_churn_flag
        , c.ltm_customer_churn_flag
        , c.ytd_customer_churn_flag

        -- MONTHLY FLAGS
        , CASE
            WHEN
                lm_customer_existing_flag = 1
                AND product_start_month = p_1.month_roll
                THEN 1
            ELSE 0
        END AS lm_cross_sell_flag
        , CASE
            WHEN
                product_grew_monthly = 1
                AND lm_product_existing_flag = 1
                THEN 1
            ELSE 0
        END AS lm_upsell_flag
        , CASE
            WHEN
                product_declined_monthly = 1
                AND lm_product_existing_flag = 1
                THEN 1
            ELSE 0
        END AS lm_downsell_flag

        -- QUARTERLY FLAGS
        , CASE
            WHEN
                l_3_m_customer_existing_flag = 1
                AND l_3_m_product_existing_flag = 0
                THEN 1
            ELSE 0
        END AS l_3_m_cross_sell_flag
        , CASE
            WHEN
                product_grew_quarterly = 1
                AND l_3_m_product_existing_flag = 1
                THEN 1
            ELSE 0
        END AS l_3_m_upsell_flag
        , CASE
            WHEN
                product_declined_quarterly = 1
                AND l_3_m_product_existing_flag = 1
                THEN 1
            ELSE 0
        END AS l_3_m_downsell_flag

        -- YEARLY FLAGS
        , CASE
            WHEN
                ltm_customer_existing_flag = 1
                AND ltm_product_existing_flag = 0
                THEN 1
            ELSE 0
        END AS ltm_cross_sell_flag
        , CASE
            WHEN
                product_grew_yearly = 1
                AND ltm_product_existing_flag = 1
                THEN 1
            ELSE 0
        END AS ltm_upsell_flag
        , CASE
            WHEN
                product_declined_yearly = 1
                AND ltm_product_existing_flag = 1
                THEN 1
            ELSE 0
        END AS ltm_downsell_flag

        -- ytd FLAGS
        , CASE
            WHEN
                ytd_customer_existing_flag = 1
                AND ytd_product_existing_flag = 0
                THEN 1
            ELSE 0
        END AS ytd_cross_sell_flag
        , CASE
            WHEN
                product_grew_ytd = 1
                AND ytd_product_existing_flag = 1
                THEN 1
            ELSE 0
        END AS ytd_upsell_flag
        , CASE
            WHEN
                product_declined_ytd = 1
                AND ytd_product_existing_flag = 1
                THEN 1
            ELSE 0
        END AS ytd_downsell_flag

    FROM product_grew p_1

    INNER JOIN "arr_sandbox"."analysis"."customer_lifecycle_events" c
        ON
            p_1.period_revenue_key = c.customer_lifecycle_events_key
            AND p_1.month_roll = c.month_roll
    INNER JOIN "arr_sandbox"."analysis"."customer_product_lifecycle_events" p_2
        ON
            p_1.period_revenue_key = p_2.customer_product_lifecycle_events_key
            AND p_1.month_roll = p_2.month_roll
    INNER JOIN "arr_sandbox"."analysis"."customer_product_contract" p_3
        ON
            p_1.customer_level_1 = p_3.customer_level_1
            AND p_1.customer_level_2 = p_3.customer_level_2
            AND p_1.customer_level_3 = p_3.customer_level_3
            AND p_1.customer_level_4 = p_3.customer_level_4
            AND p_1.customer_level_5 = p_3.customer_level_5
            AND p_1.customer_level_6 = p_3.customer_level_6
            AND p_1.customer_level_7 = p_3.customer_level_7
            AND p_1.customer_level_8 = p_3.customer_level_8
            AND p_1.customer_level_9 = p_3.customer_level_9
            AND
            p_1.product_level_1 = p_3.product_level_1
            AND p_1.product_level_2 = p_3.product_level_2
)

-- Logic for Intermittent_churn , Winback , deactivation, reactivaiton helper columns

, find_next_nonzero_month AS (

    SELECT
        current_plan.customer_level_1
        , current_plan.month_roll
        , MIN(next_plan.month_roll) AS next_nonzero_month -- finding next non zero month, so can get possible winback dates

    FROM
        ranked_product current_plan

    LEFT JOIN
        ranked_product next_plan
        ON
            current_plan.customer_level_1 = next_plan.customer_level_1
            AND next_plan.arr != 0
            AND current_plan.month_roll < next_plan.month_roll
    WHERE
        current_plan.lm_customer_new_flag = 0
    GROUP BY
        current_plan.customer_level_1
        , current_plan.month_roll
)

, find_prev_nonzero_month AS (

    SELECT DISTINCT

        current_plan.customer_level_1
        , current_plan.month_roll
        , MAX(prev_plan.month_roll) AS prev_nonzero_month  -- will get possible intermittent churn dates

    FROM
        ranked_product current_plan
    LEFT JOIN
        ranked_product prev_plan
        ON
            current_plan.customer_level_1 = prev_plan.customer_level_1
            AND prev_plan.arr != 0
            AND current_plan.month_roll > prev_plan.month_roll
    GROUP BY
        current_plan.customer_level_1
        , current_plan.month_roll
)

, find_next_nonzero_month_l_3_m AS (

    SELECT

        current_plan.customer_level_1
        , current_plan.month_roll
        , MIN(next_plan.month_roll) AS next_nonzero_month_l_3_m -- finding next non zero month, so can get possible winback dates
    FROM
        ranked_product current_plan
    LEFT JOIN
        ranked_product next_plan
        ON
            current_plan.customer_level_1 = next_plan.customer_level_1
            AND next_plan.arr_l_3_m != 0
            AND current_plan.month_roll < next_plan.month_roll
    WHERE
        current_plan.l_3_m_customer_new_flag = 0
    GROUP BY
        current_plan.customer_level_1
        , current_plan.month_roll
)

, find_prev_nonzero_month_l_3_m AS (

    SELECT DISTINCT

        current_plan.customer_level_1
        , current_plan.month_roll
        , MAX(prev_plan.month_roll) AS prev_nonzero_month_l_3_m  -- will get possible intermittent churn dates

    FROM ranked_product current_plan

    LEFT JOIN
        ranked_product prev_plan
        ON
            current_plan.customer_level_1 = prev_plan.customer_level_1
            AND prev_plan.arr_l_3_m != 0
            AND current_plan.month_roll > prev_plan.month_roll

    GROUP BY
        current_plan.customer_level_1
        , current_plan.month_roll
)

, find_next_nonzero_month_ltm AS (

    SELECT

        current_plan.customer_level_1
        , current_plan.month_roll
        , MIN(next_plan.month_roll) AS next_nonzero_month_ltm -- finding next non zero month, so can get possible winback dates

    FROM ranked_product current_plan

    LEFT JOIN
        ranked_product next_plan
        ON
            current_plan.customer_level_1 = next_plan.customer_level_1
            AND next_plan.arr_ltm != 0
            AND current_plan.month_roll < next_plan.month_roll

    WHERE current_plan.ltm_customer_new_flag = 0

    GROUP BY
        current_plan.customer_level_1
        , current_plan.month_roll
)

, find_prev_nonzero_month_ltm AS (

    SELECT DISTINCT

        current_plan.customer_level_1
        , current_plan.month_roll
        , MAX(prev_plan.month_roll) AS prev_nonzero_month_ltm  -- will get possible intermittent churn dates

    FROM
        ranked_product current_plan
    LEFT JOIN
        ranked_product prev_plan
        ON
            current_plan.customer_level_1 = prev_plan.customer_level_1
            AND prev_plan.arr_ltm != 0
            AND current_plan.month_roll > prev_plan.month_roll
    GROUP BY
        current_plan.customer_level_1
        , current_plan.month_roll
)

, find_next_nonzero_month_ytd AS (

    SELECT

        current_plan.customer_level_1
        , current_plan.month_roll
        , MIN(next_plan.month_roll) AS next_nonzero_month_ytd -- finding next non zero month, so can get possible winback dates

    FROM
        ranked_product current_plan
    LEFT JOIN
        ranked_product next_plan
        ON
            current_plan.customer_level_1 = next_plan.customer_level_1
            AND next_plan.arr_ytd != 0
            AND current_plan.month_roll < next_plan.month_roll
    WHERE
        current_plan.ytd_customer_new_flag = 0
    GROUP BY
        current_plan.customer_level_1
        , current_plan.month_roll
)

, find_prev_nonzero_month_ytd AS (

    SELECT DISTINCT

        current_plan.customer_level_1
        , current_plan.month_roll
        , MAX(prev_plan.month_roll) AS prev_nonzero_month_ytd  -- will get possible intermittent churn dates

    FROM
        ranked_product current_plan
    LEFT JOIN
        ranked_product prev_plan
        ON
            current_plan.customer_level_1 = prev_plan.customer_level_1
            AND prev_plan.arr_ytd != 0
            AND current_plan.month_roll > prev_plan.month_roll
    GROUP BY
        current_plan.customer_level_1
        , current_plan.month_roll
)

, pre_final AS (

    SELECT
        rp.*
        , fn.next_nonzero_month
        , fp.prev_nonzero_month
        , fn_3_m.next_nonzero_month_l_3_m
        , fp_3_m.prev_nonzero_month_l_3_m
        , fnm.next_nonzero_month_ltm
        , fpm.prev_nonzero_month_ltm
        , fntd.next_nonzero_month_ytd
        , fptd.prev_nonzero_month_ytd

    FROM
        ranked_product rp                                    -- will get possible dates of churn winback in one table
    LEFT JOIN
        find_next_nonzero_month fn
        ON
            rp.customer_level_1 = fn.customer_level_1
            AND rp.month_roll = fn.month_roll
    LEFT JOIN
        find_prev_nonzero_month fp
        ON
            rp.customer_level_1 = fp.customer_level_1
            AND rp.month_roll = fp.month_roll
    LEFT JOIN
        find_next_nonzero_month_l_3_m fn_3_m
        ON
            rp.customer_level_1 = fn_3_m.customer_level_1
            AND rp.month_roll = fn_3_m.month_roll
    LEFT JOIN
        find_prev_nonzero_month_l_3_m fp_3_m
        ON
            rp.customer_level_1 = fp_3_m.customer_level_1
            AND rp.month_roll = fp_3_m.month_roll
    LEFT JOIN
        find_next_nonzero_month_ltm fnm
        ON
            rp.customer_level_1 = fnm.customer_level_1
            AND rp.month_roll = fnm.month_roll
    LEFT JOIN
        find_prev_nonzero_month_ltm fpm
        ON
            rp.customer_level_1 = fpm.customer_level_1
            AND rp.month_roll = fpm.month_roll

    LEFT JOIN
        find_next_nonzero_month_ytd fntd
        ON
            rp.customer_level_1 = fntd.customer_level_1
            AND rp.month_roll = fntd.month_roll

    LEFT JOIN
        find_prev_nonzero_month_ytd fptd
        ON
            rp.customer_level_1 = fptd.customer_level_1
            AND rp.month_roll = fptd.month_roll
)

, customer_product_revenue_events AS (

    SELECT
        *
        --creating a flag column bsaed on conditions with date difference
        , CASE
            WHEN
                (SUM(arr) OVER (PARTITION BY customer_level_1, month_roll)) = 0
                AND DATEDIFF(MONTH, CAST(prev_nonzero_month AS DATE), CAST(next_nonzero_month AS DATE)) - 2 = 1
                THEN 1
            ELSE 0
        END AS deactivation_helper
        , CASE
            WHEN
                (SUM(arr) OVER (PARTITION BY customer_level_1, month_roll)) != 0
                AND DATEDIFF(MONTH, CAST(prev_nonzero_month AS DATE), CAST(month_roll AS DATE)) - 2 = 1
                THEN 1
            ELSE 0
        END AS reactivation_helper
        , CASE
            WHEN
                (SUM(arr) OVER (PARTITION BY customer_level_1, month_roll)) = 0
                AND DATEDIFF(MONTH, CAST(prev_nonzero_month AS DATE), CAST(next_nonzero_month AS DATE)) - 1 > 3
                THEN 1
            ELSE 0
        END AS intermittent_churn_helper
        , CASE
            WHEN
                (SUM(arr) OVER (PARTITION BY customer_level_1, month_roll)) != 0
                AND DATEDIFF(MONTH, CAST(prev_nonzero_month AS DATE), CAST(month_roll AS DATE)) - 1 > 3
                THEN 1
            ELSE 0
        END AS winback_helper
        , CASE
            WHEN
                (SUM(arr) OVER (PARTITION BY customer_level_1, month_roll)) != 0
                AND (SUM(arr_l_3_m) OVER (PARTITION BY customer_level_1, month_roll)) = 0
                AND DATEDIFF(MONTH, CAST(prev_nonzero_month_l_3_m AS DATE), CAST(next_nonzero_month_l_3_m AS DATE)) - 1 > 3
                THEN 1
            ELSE 0
        END AS l_3_m_winback_helper
        , CASE
            WHEN
                (SUM(arr) OVER (PARTITION BY customer_level_1, month_roll)) != 0
                AND (SUM(arr_l_3_m) OVER (PARTITION BY customer_level_1, month_roll)) = 0
                AND DATEDIFF(MONTH, CAST(prev_nonzero_month_l_3_m AS DATE), CAST(next_nonzero_month_l_3_m AS DATE)) - 2 = 1
                THEN 1
            ELSE 0
        END AS l_3_m_reactivation_helper

        , CASE
            WHEN
                (SUM(arr) OVER (PARTITION BY customer_level_1, month_roll)) != 0
                AND (SUM(arr_ltm) OVER (PARTITION BY customer_level_1, month_roll)) = 0
                AND DATEDIFF(MONTH, CAST(prev_nonzero_month_ltm AS DATE), CAST(next_nonzero_month_ltm AS DATE)) - 1 > 3
                THEN 1
            ELSE 0
        END AS ltm_winback_helper

        , CASE
            WHEN
                (SUM(arr) OVER (PARTITION BY customer_level_1, month_roll)) != 0
                AND (SUM(arr_ltm) OVER (PARTITION BY customer_level_1, month_roll)) = 0
                AND DATEDIFF(MONTH, CAST(prev_nonzero_month_ltm AS DATE), CAST(next_nonzero_month_ltm AS DATE)) - 2 = 1
                THEN 1
            ELSE 0
        END AS ltm_reactivation_helper

        , CASE
            WHEN
                (SUM(arr) OVER (PARTITION BY customer_level_1, month_roll)) != 0
                AND (SUM(arr_ytd) OVER (PARTITION BY customer_level_1, month_roll)) = 0
                AND DATEDIFF(MONTH, CAST(prev_nonzero_month_ytd AS DATE), CAST(next_nonzero_month_ytd AS DATE)) - 1 > 3
                THEN 1
            ELSE 0
        END AS ytd_winback_helper

        , CASE
            WHEN
                (SUM(arr) OVER (PARTITION BY customer_level_1, month_roll)) != 0
                AND (SUM(arr_ytd) OVER (PARTITION BY customer_level_1, month_roll)) = 0
                AND DATEDIFF(MONTH, CAST(prev_nonzero_month_ytd AS DATE), CAST(next_nonzero_month_ytd AS DATE)) - 2 = 1
                THEN 1
            ELSE 0
        END AS ytd_reactivation_helper

    FROM pre_final

)

SELECT

    period_revenue_key AS customer_product_revenue_events_key
    , customer_key
    , customer_level_1
    , customer_level_2
    , customer_level_3
    , customer_level_4
    , customer_level_5
    , customer_level_6
    , customer_level_7
    , customer_level_8
    , customer_level_9
    , customer_name
    , customer_region
    , product_key
    , product_level_1
    , product_level_2
    , other_key
    , month_roll
    , winback_helper
    , deactivation_helper
    , reactivation_helper
    , intermittent_churn_helper

    , l_3_m_reactivation_helper
    , l_3_m_winback_helper
    , ltm_reactivation_helper
    , ltm_winback_helper
    , ytd_reactivation_helper
    , ytd_winback_helper
    , CASE
        WHEN
            winback_helper = 0 AND deactivation_helper = 0
            AND reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN lm_cross_sell_flag
        ELSE 0
    END                AS lm_cross_sell_flag
    , CASE
        WHEN
            winback_helper = 0 AND deactivation_helper = 0
            AND reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN lm_upsell_flag
        ELSE 0
    END                AS lm_upsell_flag
    , CASE
        WHEN
            winback_helper = 0 AND deactivation_helper = 0
            AND reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN lm_downsell_flag
        ELSE 0
    END                AS lm_downsell_flag
    , CASE
        WHEN
            l_3_m_winback_helper = 0 AND deactivation_helper = 0
            AND l_3_m_reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN l_3_m_cross_sell_flag
        ELSE 0
    END                AS l_3_m_cross_sell_flag
    , CASE
        WHEN
            l_3_m_winback_helper = 0 AND deactivation_helper = 0
            AND l_3_m_reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN l_3_m_upsell_flag
        ELSE 0
    END                AS l_3_m_upsell_flag
    , CASE
        WHEN
            l_3_m_winback_helper = 0 AND deactivation_helper = 0
            AND l_3_m_reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN l_3_m_downsell_flag
        ELSE 0
    END                AS l_3_m_downsell_flag
    , CASE
        WHEN
            ltm_winback_helper = 0 AND deactivation_helper = 0
            AND ltm_reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN ltm_cross_sell_flag
        ELSE 0
    END                AS ltm_cross_sell_flag
    , CASE
        WHEN
            ltm_winback_helper = 0 AND deactivation_helper = 0
            AND ltm_reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN ltm_upsell_flag
        ELSE 0
    END                AS ltm_upsell_flag
    , CASE
        WHEN
            ltm_winback_helper = 0 AND deactivation_helper = 0
            AND ltm_reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN ltm_downsell_flag
        ELSE 0
    END                AS ltm_downsell_flag
    , CASE
        WHEN
            ytd_winback_helper = 0 AND deactivation_helper = 0
            AND ytd_reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN ytd_cross_sell_flag
        ELSE 0
    END                AS ytd_cross_sell_flag
    , CASE
        WHEN
            ytd_winback_helper = 0 AND deactivation_helper = 0
            AND ytd_reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN ytd_upsell_flag
        ELSE 0
    END                AS ytd_upsell_flag
    , CASE
        WHEN
            ytd_winback_helper = 0 AND deactivation_helper = 0
            AND ytd_reactivation_helper = 0 AND intermittent_churn_helper = 0
            THEN ytd_downsell_flag
        ELSE 0
    END                AS ytd_downsell_flag

FROM
    customer_product_revenue_events


##### **delta_revenue**

In [ ]:
%%sql
DROP TABLE IF EXISTS analysis.delta_revenue;
CREATE TABLE analysis.delta_revenue AS
/* This stored procedure calculates revenue deltas by applying flags for changes like acquisition, churn, cross-sell, upsell, and downsell over various time periods(monthly, quarterly, last 12 months, and year-to-date), using joins between revenue data and customer and product lifecycle tables.*/

WITH get_arr_business_flag AS (

    SELECT

        p_1.period_revenue_key
        , p_1.customer_key
        , p_1.product_key
        , p_1.other_key
        , p_1.customer_level_1
        , p_1.customer_level_2
        , p_1.customer_level_3
        , p_1.customer_level_4
        , p_1.customer_level_5
        , p_1.customer_level_6
        , p_1.customer_level_7
        , p_1.customer_level_8
        , p_1.customer_level_9
        , p_1.product_level_1
        , p_1.product_level_2
        , p_1.month_roll
        , p_1.arr
        , p_1.arr_lm
        , p_1.arr_l_3_m
        , p_1.arr_ltm
        , p_1.arr_ytd
        , p_1.arr_lm_delta
        , p_1.arr_l_3_m_delta
        , p_1.arr_ltm_delta
        , p_1.arr_ytd_delta

        -- PRICE VOLUME 
        -- uncomment below fields to find the price volume difference

        -- , p1.percentage_price_change_lm
        -- , p1.percentage_price_change_l3m
        -- , p1.percentage_price_change_ltm
        -- , p1.percentage_price_change_ytd

        , c.lm_customer_new_flag
        , c.l_3_m_customer_new_flag
        , c.ltm_customer_new_flag
        , c.ytd_customer_new_flag

        , c.lm_customer_churn_flag
        , c.l_3_m_customer_churn_flag
        , c.ltm_customer_churn_flag
        , c.ytd_customer_churn_flag

        -- , c.lm_customer_existing_flag
        -- , c.l3m_customer_existing_flag
        -- , c.ltm_customer_existing_flag
        -- , c.ytd_customer_existing_flag

        , p_2.lm_product_churn_flag
        , p_2.l_3_m_product_churn_flag
        , p_2.ltm_product_churn_flag
        , p_2.ytd_product_churn_flag

        -- , p2.lm_product_existing_flag
        -- , p2.l3m_product_existing_flag
        -- , p2.ltm_product_existing_flag
        -- , p2.ytd_product_existing_flag

        , b.winback_helper
        , b.deactivation_helper
        , b.reactivation_helper
        , b.intermittent_churn_helper
        , b.lm_cross_sell_flag
        , b.lm_upsell_flag
        , b.lm_downsell_flag

        , b.l_3_m_winback_helper
        , b.l_3_m_reactivation_helper
        , b.l_3_m_cross_sell_flag
        , b.l_3_m_upsell_flag
        , b.l_3_m_downsell_flag

        , b.ltm_winback_helper
        , b.ltm_reactivation_helper
        , b.ltm_cross_sell_flag
        , b.ltm_upsell_flag
        , b.ltm_downsell_flag

        , ytd_winback_helper
        , ytd_reactivation_helper
        , b.ytd_cross_sell_flag
        , b.ytd_upsell_flag
        , b.ytd_downsell_flag

    FROM
        "arr_sandbox"."analysis"."period_revenue" p_1

    INNER JOIN "arr_sandbox"."analysis"."customer_lifecycle_events" c
        ON
            p_1.period_revenue_key = c.customer_lifecycle_events_key
            AND p_1.month_roll = c.month_roll
    INNER JOIN "arr_sandbox"."analysis"."customer_product_lifecycle_events" p_2
        ON
            p_1.period_revenue_key = p_2.customer_product_lifecycle_events_key
            AND p_1.month_roll = p_2.month_roll
    INNER JOIN "arr_sandbox"."analysis"."customer_product_revenue_events" b
        ON
            p_1.period_revenue_key = b.customer_product_revenue_events_key
            AND p_1.month_roll = b.month_roll
)

, filling_delta AS (

    SELECT

        period_revenue_key
        , customer_key
        , product_key
        , other_key
        , customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , product_level_1
        , product_level_2
        , month_roll

        -- MONTHLY DELTAS
        , CASE
            WHEN lm_customer_new_flag = 1
                THEN arr
            ELSE 0
        END AS lm_delta_customer_new

        , CASE
            WHEN lm_customer_churn_flag = 1
                THEN -arr_lm
            ELSE 0
        END AS lm_delta_customer_churn

        , CASE
            WHEN deactivation_helper = 1
                THEN -arr_lm
            ELSE 0
        END AS lm_deactivation

        , CASE
            WHEN reactivation_helper = 1
                THEN arr
            ELSE 0
        END AS lm_reactivation

        , CASE
            WHEN intermittent_churn_helper = 1
                THEN -arr_lm
            ELSE 0
        END AS lm_intermittent_churn

        , CASE
            WHEN winback_helper = 1
                THEN arr
            ELSE 0
        END AS lm_winback

        , CASE
            WHEN lm_cross_sell_flag = 1
                THEN arr
            ELSE 0
        END AS lm_delta_cross_sell

        , CASE
            WHEN
                deactivation_helper = 0
                AND intermittent_churn_helper = 0
                AND lm_product_churn_flag = 1
                THEN -arr_lm
            ELSE 0
        END AS lm_delta_downgrade

        , CASE
            WHEN lm_upsell_flag = 1
                THEN arr_lm_delta
            ELSE 0
        END AS lm_delta_upsell

        , CASE
            WHEN lm_downsell_flag = 1
                THEN arr_lm_delta
            ELSE 0
        END AS lm_delta_downsell

        -- QUARTERLY DELTAS
        , CASE
            WHEN l_3_m_customer_new_flag = 1
                THEN arr
            ELSE 0
        END AS l_3_m_delta_customer_new

        , CASE
            WHEN l_3_m_customer_churn_flag = 1
                THEN -arr_l_3_m
            ELSE 0
        END AS l_3_m_delta_customer_churn

        , CASE
            WHEN deactivation_helper = 1
                THEN -arr_l_3_m
            ELSE 0
        END AS l_3_m_deactivation

        , CASE
            WHEN intermittent_churn_helper = 1
                THEN -arr_l_3_m
            ELSE 0
        END AS l_3_m_intermittent_churn

        , CASE
            WHEN l_3_m_reactivation_helper = 1
                THEN arr
            ELSE 0
        END AS l_3_m_reactivation

        , CASE
            WHEN l_3_m_winback_helper = 1
                THEN arr
            ELSE 0
        END AS l_3_m_winback

        , CASE
            WHEN l_3_m_cross_sell_flag = 1
                THEN arr
            ELSE 0
        END AS l_3_m_delta_cross_sell

        , CASE
            WHEN
                deactivation_helper = 0
                AND intermittent_churn_helper = 0 AND l_3_m_product_churn_flag = 1
                THEN -arr_l_3_m
            ELSE 0
        END AS l_3_m_delta_downgrade

        , CASE
            WHEN l_3_m_upsell_flag = 1
                THEN arr_l_3_m_delta
            ELSE 0
        END AS l_3_m_delta_upsell

        , CASE
            WHEN l_3_m_downsell_flag = 1
                THEN arr_l_3_m_delta
            ELSE 0
        END AS l_3_m_delta_downsell

        -- YEARLY DELTAS
        , CASE
            WHEN ltm_customer_new_flag = 1
                THEN arr
            ELSE 0
        END AS ltm_delta_customer_new

        , CASE
            WHEN ltm_customer_churn_flag = 1
                THEN -arr_ltm
            ELSE 0
        END AS ltm_delta_customer_churn

        , CASE
            WHEN deactivation_helper = 1
                THEN -arr_ltm
            ELSE 0
        END AS ltm_deactivation

        , CASE
            WHEN intermittent_churn_helper = 1
                THEN -arr_ltm
            ELSE 0
        END AS ltm_intermittent_churn

        , CASE
            WHEN ltm_reactivation_helper = 1
                THEN arr
            ELSE 0
        END AS ltm_reactivation

        , CASE
            WHEN ltm_winback_helper = 1
                THEN arr
            ELSE 0
        END AS ltm_winback

        , CASE
            WHEN ltm_cross_sell_flag = 1
                THEN arr
            ELSE 0
        END AS ltm_delta_cross_sell

        , CASE
            WHEN
                deactivation_helper = 0
                AND intermittent_churn_helper = 0 AND ltm_product_churn_flag = 1
                THEN -arr_ltm
            ELSE 0
        END AS ltm_delta_downgrade

        , CASE
            WHEN ltm_upsell_flag = 1
                THEN arr_ltm_delta
            ELSE 0
        END AS ltm_delta_upsell

        , CASE
            WHEN ltm_downsell_flag = 1
                THEN arr_ltm_delta
            ELSE 0
        END AS ltm_delta_downsell

        -- YTD DELTAS
        , CASE
            WHEN ytd_customer_new_flag = 1
                THEN arr
            ELSE 0
        END AS ytd_delta_customer_new

        , CASE
            WHEN deactivation_helper = 1
                THEN -arr_ytd
            ELSE 0
        END AS ytd_deactivation

        , CASE
            WHEN ytd_customer_churn_flag = 1
                THEN -arr_ytd
            ELSE 0
        END AS ytd_delta_customer_churn

        , CASE
            WHEN intermittent_churn_helper = 1
                THEN -arr_ytd
            ELSE 0
        END AS ytd_intermittent_churn

        , CASE
            WHEN ytd_reactivation_helper = 1
                THEN arr
            ELSE 0
        END AS ytd_reactivation

        , CASE
            WHEN ytd_winback_helper = 1
                THEN arr
            ELSE 0
        END AS ytd_winback

        , CASE
            WHEN ytd_cross_sell_flag = 1
                THEN arr
            ELSE 0
        END AS ytd_delta_cross_sell

        , CASE
            WHEN
                deactivation_helper = 0
                AND intermittent_churn_helper = 0 AND ytd_product_churn_flag = 1
                THEN -arr_ytd
            ELSE 0
        END AS ytd_delta_downgrade

        , CASE
            WHEN ytd_upsell_flag = 1
                THEN arr_ytd_delta
            ELSE 0
        END AS ytd_delta_upsell

        , CASE
            WHEN ytd_downsell_flag = 1
                THEN arr_ytd_delta
            ELSE 0
        END AS ytd_delta_downsell

        -- PRICE VOLUME
    -- uncomment below CASE STATEMENTS to find the price volume difference

    -- MONTHLY
    -- , CASE  
    --     WHEN lm_upsell_flag = 1
    --     THEN arr_lm_delta * percentage_price_change_lm
    --     ELSE 0
    -- END AS lm_delta_price_upsell

    -- , CASE  
    --     WHEN lm_upsell_flag = 1
    --     THEN arr_lm_delta * (1 - percentage_price_change_lm)
    --     ELSE 0
    -- END AS lm_delta_volume_upsell

    -- , CASE  
    --     WHEN lm_downsell_flag = 1
    --     THEN arr_lm_delta * percentage_price_change_lm
    --     ELSE 0
    -- END AS lm_delta_price_downsell

    -- , CASE  
    --     WHEN lm_downsell_flag = 1
    --     THEN arr_lm_delta * (1 - percentage_price_change_lm)
    --     ELSE 0
    -- END AS lm_delta_volume_downsell

    -- -- QUARTERLY
    -- , CASE  
    --     WHEN l3m_upsell_flag = 1
    --     THEN arr_l3m_delta * percentage_price_change_l3m
    --     ELSE 0
    -- END AS l3m_delta_price_upsell

    -- , CASE  
    --     WHEN l3m_upsell_flag = 1
    --     THEN arr_l3m_delta * (1 - percentage_price_change_l3m)
    --     ELSE 0
    -- END AS l3m_delta_volume_upsell

    -- , CASE  
    --     WHEN l3m_downsell_flag = 1
    --     THEN arr_l3m_delta * percentage_price_change_l3m
    --     ELSE 0
    -- END AS l3m_delta_price_downsell

    -- , CASE  
    --     WHEN l3m_downsell_flag = 1
    --     THEN arr_l3m_delta * (1 - percentage_price_change_l3m)
    --     ELSE 0
    -- END AS l3m_delta_volume_downsell

    -- -- YEARLY

    -- , CASE  
    --     WHEN ltm_upsell_flag = 1
    --     THEN arr_ltm_delta * percentage_price_change_ltm
    --     ELSE 0
    -- END AS ltm_delta_price_upsell

    -- , CASE  
    --     WHEN ltm_upsell_flag = 1
    --     THEN arr_ltm_delta * (1 - percentage_price_change_ltm)
    --     ELSE 0
    -- END AS ltm_delta_volume_upsell

    -- , CASE  
    --     WHEN ltm_downsell_flag = 1
    --     THEN arr_ltm_delta * percentage_price_change_ltm
    --     ELSE 0
    -- END AS ltm_delta_price_downsell

    -- , CASE  
    --     WHEN ltm_downsell_flag = 1
    --     THEN arr_ltm_delta * (1 - percentage_price_change_ltm)
    --     ELSE 0
    -- END AS ltm_delta_volume_downsell

    -- -- YTD    

    -- , CASE  
    --     WHEN ytd_upsell_flag = 1
    --     THEN arr_ytd_delta * percentage_price_change_ytd
    --     ELSE 0
    -- END AS ytd_delta_price_upsell

    -- , CASE  
    --     WHEN ytd_upsell_flag = 1
    --     THEN arr_ytd_delta * (1 - percentage_price_change_ytd)
    --     ELSE 0
    -- END AS ytd_delta_volume_upsell

    -- , CASE  
    --     WHEN ytd_downsell_flag = 1
    --     THEN arr_ytd_delta * percentage_price_change_ytd
    --     ELSE 0
    -- END AS ytd_delta_price_downsell

    -- , CASE  
    --     WHEN ytd_downsell_flag=1
    --     THEN arr_ytd_delta * (1 - percentage_price_change_ytd)
    --     ELSE 0
    -- END AS ytd_delta_volume_downsell

    FROM get_arr_business_flag
)

SELECT

    period_revenue_key AS delta_revenue_key
    , month_roll
    , customer_key
    , product_key
    , other_key
    , customer_level_1
    , customer_level_2
    , customer_level_3
    , customer_level_4
    , customer_level_5
    , customer_level_6
    , customer_level_7
    , customer_level_8
    , customer_level_9
    , product_level_1
    , product_level_2

    , lm_delta_customer_new
    , lm_delta_customer_churn
    , lm_deactivation
    , lm_reactivation
    , lm_intermittent_churn
    , lm_winback
    , lm_delta_cross_sell
    , lm_delta_downgrade
    , lm_delta_upsell
    -- , lm_delta_price_upsell
    -- , lm_delta_volume_upsell
    , lm_delta_downsell
    -- , lm_delta_price_downsell
    -- , lm_delta_volume_downsell

    , l_3_m_delta_customer_new
    , l_3_m_delta_customer_churn
    , l_3_m_deactivation
    , l_3_m_reactivation
    , l_3_m_intermittent_churn
    , l_3_m_winback
    , l_3_m_delta_cross_sell
    , l_3_m_delta_downgrade
    , l_3_m_delta_upsell
    -- , l3m_delta_price_upsell
    -- , l3m_delta_volume_upsell
    , l_3_m_delta_downsell
    -- , l3m_delta_price_downsell
    -- , l3m_delta_volume_downsell

    , ltm_delta_customer_new
    , ltm_delta_customer_churn
    , ltm_deactivation
    , ltm_reactivation
    , ltm_intermittent_churn
    , ltm_winback
    , ltm_delta_cross_sell
    , ltm_delta_downgrade
    , ltm_delta_upsell
    -- , ltm_delta_price_upsell
    -- , ltm_delta_volume_upsell
    , ltm_delta_downsell
    -- , ltm_delta_price_downsell
    -- , ltm_delta_volume_downsell

    , ytd_delta_customer_new
    , ytd_delta_customer_churn
    , ytd_reactivation
    , ytd_deactivation
    , ytd_intermittent_churn
    , ytd_winback
    , ytd_delta_cross_sell
    , ytd_delta_downgrade
    , ytd_delta_upsell
    -- , ytd_delta_price_upsell
    -- , ytd_delta_volume_upsell
    , ytd_delta_downsell
    -- , ytd_delta_price_downsell
    -- , ytd_delta_volume_downsell

FROM filling_delta


##### **monthly_revenue**

In [ ]:
%%sql
DROP TABLE IF EXISTS analysis.monthly_revenue;
CREATE TABLE analysis.monthly_revenue AS
/* This model processes revenue data by joining it with customer contract information to calculate and aggregate ARR (Annual Recurring Revenue) across different months.*/

WITH date_joins AS (

    SELECT

        -- Joining Customer and Revenue Data
        c.*
        , p.*
        , o.*
        , r.revenue_key
        , r.revenue_type
        , r.month
        , r.revenue
        , r.mrr
        , r.volume
        , MIN(month) OVER (PARTITION BY revenue_key) AS segment_start_month
        , MAX(month) OVER (PARTITION BY revenue_key) AS segment_end_month

    FROM "arr_sandbox"."datamart"."fact_revenue" r
    LEFT JOIN
        "arr_sandbox"."datamart"."dim_customer" c
        ON r.customer_key = c.customer_key
    LEFT JOIN
        "arr_sandbox"."datamart"."dim_product" p
        ON r.product_key = p.product_key
    LEFT JOIN
        "arr_sandbox"."datamart"."dim_other" o
        ON r.other_key = o.other_key
    WHERE
        r.revenue <> 0.00
)

-- Filling in the gaps for each customer with 0 revenue whenever a record of revenue for a customer on a month is not available
, date_scaffolding AS (

    SELECT

        revenue_key
        , revenue_type
        , customer_key
        , customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , customer_name
        , customer_region
        , product_key
        , product_level_1
        , product_level_2
        , other_key
        , other_dim_1
        , other_dim_2
        , c.month_roll
        , CASE
            WHEN
                c.month_roll > d.month
                OR c.month_roll <> d.month
                THEN 0
            ELSE d.volume
        END AS volume
        , CASE
            WHEN
                c.month_roll > d.month
                OR c.month_roll <> d.month
                THEN 0
            ELSE d.mrr
        END AS mrr

    FROM "arr_sandbox"."datamart"."dim_calendar" c

    INNER JOIN date_joins d
        ON
            c.month_roll <= DATEADD(MONTH, 12, d.segment_end_month)
            AND c.month_roll >= d.segment_start_month
)

-- Create monthly_revenue table
, aggregated_revenue AS (

    SELECT

        revenue_key                                           AS monthly_revenue_key
        , revenue_type
        , customer_key
        , customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , customer_name
        , customer_region
        , product_key
        , product_level_1
        , product_level_2
        , other_key
        , other_dim_1
        , other_dim_2
        , month_roll
        , SUM(mrr)                                            AS mrr
        , SUM(volume)                                         AS volume
        -- Add 1 back to YTD year start here so YTD start aligns with month selected i.e. 4 = start in April
        , DATEPART(MONTH, DATEADD(MONTH, -4 + 1, month_roll)) AS ytd_helper
    FROM
        date_scaffolding
    GROUP BY
        revenue_key
        , customer_key
        , customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , customer_name
        , customer_region
        , product_key
        , product_level_1
        , product_level_2
        , other_key
        , other_dim_1
        , other_dim_2
        , month_roll
        , revenue_type
)

, churn_month AS (

    SELECT
        customer_key
        , product_key
        , MAX(month_roll) AS product_churn_month
    FROM
        aggregated_revenue
    WHERE
        mrr <> 0.0
    GROUP BY
        customer_key
        , product_key
)

-- Create monthly_revenue table

SELECT
    a.*
    , CASE
        WHEN a.revenue_type = 1 OR a.revenue_type = 'Recurring' THEN mrr * 12
        WHEN month_roll <= product_churn_month
            THEN SUM(mrr) OVER (
                PARTITION BY monthly_revenue_key
                ORDER BY month_roll
                ROWS BETWEEN 11 PRECEDING AND CURRENT ROW
            )
        ELSE 0
    END AS arr

FROM
    aggregated_revenue a
LEFT JOIN
    churn_month c
    ON
        a.customer_key = c.customer_key
        AND a.product_key = c.product_key


##### **period_revenue**

In [ ]:
%%sql
DROP TABLE IF EXISTS analysis.period_revenue;
CREATE TABLE analysis.period_revenue AS
/* This stored procedure calculates ARR changes over different periods (monthly, quarterly, yearly, and year-to-date) and provides insights into how revenue evolves over time.*/

WITH get_ytd_start AS (

    SELECT

        m.monthly_revenue_key
        , m.customer_key
        , m.customer_level_1
        , m.customer_level_2
        , m.customer_level_3
        , m.customer_level_4
        , m.customer_level_5
        , m.customer_level_6
        , m.customer_level_7
        , m.customer_level_8
        , m.customer_level_9
        , m.customer_name
        , m.customer_region
        , m.product_key
        , m.product_level_1
        , m.product_level_2
        , m.other_key
        , m.other_dim_1
        , m.other_dim_2
        , m.month_roll
        , m.arr
        , m.mrr
        , m.volume
        , m.ytd_helper
        , m.revenue_type
    FROM
        "arr_sandbox"."analysis"."monthly_revenue" m

)

-- Calculate revenue lags based on the above declared variables for monthly, yearly, and quarterly.

, get_revenue_lags AS (

    SELECT

        a.monthly_revenue_key
        , a.customer_key
        , a.customer_level_1
        , a.customer_level_2
        , a.customer_level_3
        , a.customer_level_4
        , a.customer_level_5
        , a.customer_level_6
        , a.customer_level_7
        , a.customer_level_8
        , a.customer_level_9
        , a.customer_name
        , a.customer_region
        , a.product_key
        , a.product_level_1
        , a.product_level_2
        , a.other_key
        , a.other_dim_1
        , a.other_dim_2
        , a.month_roll
        , a.arr
        , a.mrr
        , a.volume
        , a.revenue_type
        , COALESCE(LAG(a.arr, 1) OVER (PARTITION BY a.monthly_revenue_key ORDER BY a.month_roll), 0)  AS arr_lm   -- Monthly Revenue Lag
        , COALESCE(LAG(a.arr, 3) OVER (PARTITION BY a.monthly_revenue_key ORDER BY a.month_roll), 0)  AS arr_l_3_m  -- Quarterly Revenue Lag
        , COALESCE(LAG(a.arr, 12) OVER (PARTITION BY a.monthly_revenue_key ORDER BY a.month_roll), 0) AS arr_ltm  -- Yearly Revenue Lag
        , COALESCE(b.arr, 0)                                                                          AS arr_ytd
        -- Uncommend the below lines to get the volume lags
        -- , LAG(volume, 1, 0)    OVER (PARTITION BY monthly_revenue_key ORDER BY month_roll)     AS volume_lm,  -- Monthly Volume Lag
        -- , LAG(volume, 3, 0)  OVER (PARTITION BY monthly_revenue_key ORDER BY month_roll)     AS volume_l3m, -- Quarterly Volume Lag
        -- , LAG(volume, 12, 0)     OVER (PARTITION BY monthly_revenue_key ORDER BY month_roll)     AS volume_ltm, -- Yearly Volume Lag
        -- , LAG(volume, ytd_helper, 0)  OVER (PARTITION BY monthly_revenue_key ORDER BY month_roll)               AS volume_ytd  -- Ytd Volume Lag
    FROM get_ytd_start a

    LEFT JOIN
        get_ytd_start b
        ON
            a.customer_key = b.customer_key
            AND a.product_key = b.product_key
            AND a.month_roll = DATEADD(MONTH, a.ytd_helper, b.month_roll)
)

, get_delta_revenue AS (

    SELECT

        monthly_revenue_key
        , customer_key
        , customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , customer_name
        , customer_region
        , product_key
        , product_level_1
        , product_level_2
        , other_key
        , other_dim_1
        , other_dim_2
        , month_roll
        , revenue_type
        , mrr
        , arr
        , volume
        , arr_lm
        , arr_l_3_m
        , arr_ltm
        , arr_ytd
        -- Uncommand the lines to get the  volume lags
        -- volume_lm,
        -- volume_l3m,
        -- volume_ltm,
        -- volume_ytd,
        , arr - arr_lm    AS arr_lm_delta
        , arr - arr_l_3_m AS arr_l_3_m_delta
        , arr - arr_ltm   AS arr_ltm_delta
        , arr - arr_ytd   AS arr_ytd_delta

    FROM
        get_revenue_lags
)

, find_price_volume_deltas AS (
    SELECT
        customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , product_level_1
        , product_level_2
        , month_roll
        , revenue_type
        -- Master Product level Revenue
        , SUM(arr_lm_delta)    AS sum_arr_lm_delta
        , SUM(arr_l_3_m_delta) AS sum_arr_l_3_m_delta
        , SUM(arr_ltm_delta)   AS sum_arr_ltm_delta
        , SUM(arr_ytd_delta)   AS sum_arr_ytd_delta

    -- Uncomment the below lines to get the price volume increases
    -- -- PRICE DELTAS
    -- , CASE
    --     -- Check if both current and last month's volumes are non-zero to avoid division by zero
    --     WHEN SUM(volume) <> 0 AND SUM(volume_lm) <> 0 THEN 
    --         -- Calculate the price change (ARR per unit volume) between current and last month
    --         ((SUM(arr) / SUM(volume)) - (SUM(arr_lm) / SUM(volume_lm))) * SUM(volume)
    --     ELSE 
    --         -- If either volume is zero, set the price delta to 0
    --         0 
    -- END AS abs_price_lm_delta

    -- , CASE
    --     WHEN SUM(volume) <> 0 AND SUM(volume_l3m) <> 0 THEN 
    --         ((SUM(arr) / SUM(volume)) - (SUM(arr_l3m) / SUM(volume_l3m))) * SUM(volume)
    --     ELSE 
    --         0 
    -- END AS abs_price_l3m_delta

    -- , CASE
    --     WHEN SUM(volume) <> 0 AND SUM(volume_ltm) <> 0 THEN 
    --         ((SUM(arr) / SUM(volume)) - (SUM(arr_ltm) / SUM(volume_ltm))) * SUM(volume)
    --     ELSE 
    --         0 
    -- END AS abs_price_ltm_delta

    -- , CASE
    --     WHEN SUM(volume) <> 0 AND SUM(volume_ytd) <> 0 THEN 
    --         ((SUM(arr) / SUM(volume)) - (SUM(arr_ytd) / SUM(volume_ytd))) * SUM(volume)
    --     ELSE 
    --         0 
    -- END AS abs_price_ytd_delta

    -- -- VOLUME DELTAS
    -- , CASE
    --     WHEN SUM(volume) <> 0 AND SUM(volume_lm) <> 0 THEN 
    --         (SUM(volume) - SUM(volume_lm)) * SUM(arr_lm) / SUM(volume_lm)
    --     ELSE 
    --         SUM(arr_lm_delta) 
    -- END AS abs_volume_lm_delta

    -- , CASE
    --     WHEN SUM(volume) <> 0 AND SUM(volume_l3m) <> 0 THEN 
    --         (SUM(volume) - SUM(volume_l3m)) * SUM(arr_l3m) / SUM(volume_l3m)
    --     ELSE 
    --         SUM(arr_l3m_delta) 
    -- END AS abs_volume_l3m_delta

    -- , CASE
    --     WHEN SUM(volume) <> 0 AND SUM(volume_ltm) <> 0 THEN 
    --         (SUM(volume) - SUM(volume_ltm)) * SUM(arr_ltm) / SUM(volume_ltm)
    --     ELSE 
    --         SUM(arr_ltm_delta) 
    -- END AS abs_volume_ltm_delta

    -- , CASE
    --     WHEN SUM(volume) <> 0 AND SUM(volume_ytd) <> 0 THEN 
    --         (SUM(volume) - SUM(volume_ytd)) * SUM(arr_ytd) / SUM(volume_ytd)
    --     ELSE 
    --         SUM(arr_ytd_delta) 
    -- END AS abs_volume_ytd_delta

    FROM
        get_delta_revenue
    GROUP BY
        customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , product_level_1
        , product_level_2
        , month_roll
        , revenue_type
)

, get_percentage_change AS (
    SELECT
        customer_level_1
        , customer_level_2
        , customer_level_3
        , customer_level_4
        , customer_level_5
        , customer_level_6
        , customer_level_7
        , customer_level_8
        , customer_level_9
        , product_level_1
        , product_level_2
        , month_roll
        , revenue_type
        , sum_arr_lm_delta
        , sum_arr_l_3_m_delta
        , sum_arr_ltm_delta
        , sum_arr_ytd_delta

    -- Uncomment the lines to get the price volume increases
    -- MONTHLY
    -- , CASE 
    --     WHEN abs_price_lm_delta <> 0 THEN 1 
    --     ELSE 0 
    -- END AS price_change_lm

    -- , CASE 
    --     WHEN abs_volume_lm_delta <> 0 THEN 1 
    --     ELSE 0 
    -- END AS volume_change_lm

    -- , CASE 
    --     WHEN sum_arr_lm_delta <> 0 THEN abs_price_lm_delta / sum_arr_lm_delta 
    -- END AS percentage_price_change_lm

    -- -- QUARTERLY
    -- , CASE 
    --     WHEN abs_price_l3m_delta <> 0 THEN 1 
    --     ELSE 0 
    -- END AS price_change_l3m

    -- , CASE 
    --     WHEN abs_volume_l3m_delta <> 0 THEN 1 
    --     ELSE 0 
    -- END AS volume_change_l3m

    -- , CASE 
    --     WHEN sum_arr_l3m_delta <> 0 THEN abs_price_l3m_delta / sum_arr_l3m_delta 
    -- END AS percentage_price_change_l3m

    -- -- YEARLY
    -- , CASE 
    --     WHEN abs_price_ltm_delta <> 0 THEN 1 
    --     ELSE 0 
    -- END AS price_change_ltm

    -- , CASE 
    --     WHEN abs_volume_ltm_delta <> 0 THEN 1 
    --     ELSE 0 
    -- END AS volume_change_ltm

    -- , CASE 
    --     WHEN sum_arr_ltm_delta <> 0 THEN abs_price_ltm_delta / sum_arr_ltm_delta 
    -- END AS percentage_price_change_ltm

    -- -- YTD
    -- , CASE 
    --     WHEN abs_price_ytd_delta <> 0 THEN 1 
    --     ELSE 0 
    -- END AS price_change_ytd

    -- , CASE 
    --     WHEN abs_volume_ytd_delta <> 0 THEN 1 
    --     ELSE 0 
    -- END AS volume_change_ytd

    -- , CASE 
    --     WHEN sum_arr_ytd_delta <> 0 THEN abs_price_ytd_delta / sum_arr_ytd_delta 
    -- END AS percentage_price_change_ytd

    FROM
        find_price_volume_deltas
)

SELECT

    r.monthly_revenue_key AS period_revenue_key
    , r.customer_key
    , r.customer_level_1
    , r.customer_level_2
    , r.customer_level_3
    , r.customer_level_4
    , r.customer_level_5
    , r.customer_level_6
    , r.customer_level_7
    , r.customer_level_8
    , r.customer_level_9
    , r.customer_name
    , r.customer_region
    , r.product_key
    , r.product_level_1
    , r.product_level_2
    , r.other_key
    , r.other_dim_1
    , r.other_dim_2
    , r.month_roll
    , r.mrr
    , r.arr
    , r.volume
    , r.arr_lm
    , r.arr_l_3_m
    , r.arr_ltm
    , r.arr_ytd
    , r.arr_lm_delta
    , r.arr_l_3_m_delta
    , r.arr_ltm_delta
    , r.arr_ytd_delta
    , p.sum_arr_lm_delta
    , p.sum_arr_l_3_m_delta
    , p.sum_arr_ltm_delta
    , p.sum_arr_ytd_delta
-- Uncomment the lines to get the price volume increases
-- , p.abs_price_lm_delta
-- , p.percentage_price_change_lm
-- , p.percentage_price_change_l3m
-- , p.percentage_price_change_ltm
-- , p.percentage_price_change_ytd

FROM get_delta_revenue r
LEFT JOIN
    get_percentage_change p
    ON
        r.customer_level_1 = p.customer_level_1
        AND r.customer_level_2 = p.customer_level_2
        AND r.customer_level_3 = p.customer_level_3
        AND r.customer_level_4 = p.customer_level_4
        AND r.customer_level_5 = p.customer_level_5
        AND r.customer_level_6 = p.customer_level_6
        AND r.customer_level_7 = p.customer_level_7
        AND r.customer_level_8 = p.customer_level_8
        AND r.customer_level_9 = p.customer_level_9
        AND r.product_level_1 = p.product_level_1
        AND r.product_level_2 = p.product_level_2
        AND r.month_roll = p.month_roll
        AND r.revenue_type = p.revenue_type
